In [94]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix

In [40]:
train_dataset = pd.read_csv(r"C:/Users/Dell/Documents/intern/Codesoft/archive/Genre Classification Dataset/train_data.txt",delimiter=':::',names=['ID','TITLE','GENRE','DESCRIPTION'],engine='python')
train_dataset.head(10)
test_dataset = pd.read_csv(r"C:/Users/Dell/Documents/intern/Codesoft/archive/Genre Classification Dataset/test_data.txt",delimiter=':::',names=['ID','TITLE','DESCRIPTION'],engine='python')
test_dataset.head(10)

,ID,TITLE,DESCRIPTION
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,5,Er nu zhai (1955),Before he was known internationally as a mart...
5,6,Riddle Room (2016),Emily Burns is being held captive in a room w...
6,7,L'amica (1969),The beautiful but neglected wife of a brillia...
7,8,Ina Mina Dika (1989),Vasu Inamdar (Ina) suffers from a disorder wh...
8,9,Equinox Special: Britain's Tornados (2005),An insight into the tornados that hit Kensal ...
9,10,Press (2011),Press is a story of young people overwhelmed ...


In [41]:
print(f"Missing values in ID: {train_dataset['ID'].isnull().sum()}")
print(f"Missing values in TITLE: {train_dataset['TITLE'].isnull().sum()}")
print(f"Missing values in ID: {train_dataset['ID'].isnull().sum()}")
print(f"Missing values in ID: {train_dataset['ID'].isnull().sum()}")

Missing values in ID: 0
Missing values in TITLE: 0
Missing values in ID: 0
Missing values in ID: 0


In [93]:
x_train,x_test = train_dataset['DESCRIPTION'].astype(str).apply(lambda des:des.lower()),test_dataset['DESCRIPTION'].astype(str).apply(lambda des:des.lower())
genre_labels = [genre.split(', ') for genre in train_dataset['GENRE']]
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(genre_labels)
print(y_train.shape)

(54214, 27)


In [86]:
tfidf_vectorizer = TfidfVectorizer(max_features=12000)
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)
print(f"x_train shape: {x_train_tfidf.shape}")
print(f"x_test shape: {x_test_tfidf.shape}")

x_train shape: (54214, 12000)
x_test shape: (54200, 12000)


In [87]:
navie_bayes = MultinomialNB()
multi_output_classifier = MultiOutputClassifier(navie_bayes)
multi_output_classifier.fit(x_train_tfidf,y_train)

MultiOutputClassifier(estimator=MultinomialNB())

In [88]:
y_pred = multi_output_classifier.predict(x_test_tfidf)
test_movie_names = test_dataset['TITLE']
predicted_genres = mlb.inverse_transform(y_pred)
test_results = pd.DataFrame({'Title':test_movie_names,'Predicted Genres':predicted_genres})
test_results.head(5)

,Title,Predicted Genres
0,Edgar's Lunch (1998),()
1,La guerra de papá (1977),"( drama ,)"
2,Off the Beaten Track (2010),"( documentary ,)"
3,Meu Amigo Hindu (2015),"( drama ,)"
4,Er nu zhai (1955),()


In [89]:
test_results['Predicted Genres'] = test_results['Predicted Genres'].apply(lambda genre: 'Unknown prediction' if len(genre)==0 else genre[0])
test_results.head(5)

,Title,Predicted Genres
0,Edgar's Lunch (1998),Unknown prediction
1,La guerra de papá (1977),drama
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),Unknown prediction


In [90]:
test_sol_dataset = pd.read_csv(r"C:/Users/Dell/Documents/intern/Codesoft/archive/Genre Classification Dataset/test_data_solution.txt",delimiter=":::",names=['ID','TITLE','GENRE','DESCRIPTION'],engine='python')
test_sol_dataset = pd.DataFrame({"TITLE":test_sol_dataset["TITLE"],"GENRE":test_sol_dataset["GENRE"]})
test_sol_dataset.head(5)

,TITLE,GENRE
0,Edgar's Lunch (1998),thriller
1,La guerra de papá (1977),comedy
2,Off the Beaten Track (2010),documentary
3,Meu Amigo Hindu (2015),drama
4,Er nu zhai (1955),drama


In [96]:
accuracy = accuracy_score(test_sol_dataset["GENRE"],test_results['Predicted Genres'])
print(accuracy)

0.26339483394833946
